In [24]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error


In [24]:
%pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 12.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 12.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [269]:

def ann_system(hub1_name, hub2_name, validation_size, test_size, window_size, verbose=True, save=True):
    hub1 = pd.read_csv(f"../../data/interpolated/{hub1_name}_close_interpolated.csv")
    hub2 = pd.read_csv(f"../../data/interpolated/{hub2_name}_close_interpolated.csv")

    hub1 = hub1.rename(columns={"CLOSE": "hub1_CLOSE"})
    hub2 = hub2.rename(columns={"CLOSE": "hub2_CLOSE"})
    hub1_hub2_diff = pd.DataFrame(hub1["hub1_CLOSE"] - hub2["hub2_CLOSE"], columns=["hub1_hub2_diff"], index=hub1.index)

    # Shift columns and store in new columns for hub1, hub2, and hub1_hub2_diff
    for i in range(window_size, window_size + 6):
        hub1[f"hub1_CLOSE-{i- window_size}"] = hub1["hub1_CLOSE"].shift(i)
        hub2[f"hub2_CLOSE-{i - window_size}"] = hub2["hub2_CLOSE"].shift(i)
        hub1_hub2_diff[f"hub1_hub2_diff-{i - window_size}"] = hub1_hub2_diff["hub1_hub2_diff"].shift(i)

    # Concatenate and drop NaN rows in one step
    data = pd.concat([hub1, hub2, hub1_hub2_diff], axis=1).dropna()

    features = [
        'hub1_CLOSE-0', #'hub1_CLOSE-1', #'hub1_CLOSE-2', 'hub1_CLOSE-3', 'hub1_CLOSE-4', 'hub1_CLOSE-5',
        'hub2_CLOSE-0', #'hub2_CLOSE-1', #'hub2_CLOSE-2', 'hub2_CLOSE-3', 'hub2_CLOSE-4', 'hub2_CLOSE-5',
        'hub1_hub2_diff-0', 'hub1_hub2_diff-1', 'hub1_hub2_diff-2', 'hub1_hub2_diff-3', 'hub1_hub2_diff-4', 'hub1_hub2_diff-5',# 'hub1_hub2_diff-6'
    ]

    X = data[features].values

    y = data[['hub1_CLOSE', 'hub2_CLOSE']].values

    X_train, X_test = X[:-test_size], X[-test_size:]
    y_train, y_test = y[:-test_size], y[-test_size:]

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    #X_train, X_val = X_train[:-validation_size], X_train[-validation_size:]
    #y_train, y_val = y_train[:-validation_size], y_train[-validation_size:]

    # Build a simple ANN model
    model = Sequential([
        Dense(2, activation='relu'),
        #Dense(1, activation='linear'),
        Dense(2)  # 2 outputs for both hubs
    ])
    #lr = 0.0038336791635842195 #TTF-THE
    #lr = 0.00026670003536052503 #TTF-NBP
    #lr = 0.0018046087209813773 #THE-NBP 2
    lr = 0.003319584493448066 #THE-NBP
    #lr = 0.001
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=lr), loss='mape')

    # Train the model
    history = model.fit(X_train, y_train, epochs=25, batch_size=1, 
                        #validation_data=(X_val, y_val), 
                        shuffle=False,
                        verbose=1)

    # Evaluate the model
    test_loss = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test Loss: {test_loss}")

    # Predict on test data
    predictions = model.predict(X_test)

    # Calculate MAPE
    mape_hub1 = mean_absolute_percentage_error(y_test[:, 0], predictions[:, 0]) * 100
    print(f"MAPE for {hub1_name}: {mape_hub1:.2f}%")

    # Calculate MAPE for Hub 2
    mape_hub2 = mean_absolute_percentage_error(y_test[:, 1], predictions[:, 1]) * 100
    print(f"MAPE for {hub2_name}: {mape_hub2:.2f}%")



    return hub1, hub2

In [270]:
keras.utils.set_random_seed(42)

hub1 = "ttf"
hub2 = "nbp"
validation_size = 250
test_size = 250
window_size = 5

data = ann_system(hub1, hub2, validation_size, test_size, window_size)

(1285, 8) (1285, 2) (250, 8) (250, 2)
Epoch 1/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 2s 554us/step - loss: 10.0788
Epoch 2/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - loss: 7.7384
Epoch 3/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 546us/step - loss: 7.6237
Epoch 4/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 536us/step - loss: 7.5603
Epoch 5/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 548us/step - loss: 7.5572
Epoch 6/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 551us/step - loss: 7.4848
Epoch 7/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - loss: 7.4455
Epoch 8/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step - loss: 7.5068
Epoch 9/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 542us/step - loss: 7.4964
Epoch 10/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 547us/step - loss: 7.5807
Epoch 11/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 556us/step - loss: 7.5522
Epoch 12/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 574us/step - loss: 7.5527
Epoch 13/25
1285/1285 ━━━━━━━━━━━━━━━━━━━━ 1s 568us/step - loss: 7.6426
Epoch 14/25
1285/1285 ━━━━━━━━━━━━

In [219]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error
import optuna
keras.utils.set_random_seed(42)

hub1_name = "ttf"
hub2_name = "the"

def build_model(trial):
    model = Sequential()

    # Tune the number of hidden layers and units per layer
    #n_layers = trial.suggest_int("n_layers", 1, 3)
    n_layers = 1
    for i in range(n_layers):
        units = trial.suggest_int(f"units_{i}", 2, 2, step=1)
        #units = 1
        model.add(Dense(units, activation='relu'))

    model.add(Dense(2))  # Output layer for both hubs

    # Tune the learning rate
    lr = trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True)
    model.compile(optimizer=Adam(learning_rate=lr), loss='mape')
    return model

def load_and_prepare_data(hub1_name, hub2_name, window_size, test_size, validation_size):
    hub1 = pd.read_csv(f"../../data/interpolated/{hub1_name}_close_interpolated.csv")
    hub2 = pd.read_csv(f"../../data/interpolated/{hub2_name}_close_interpolated.csv")

    hub1 = hub1.rename(columns={"CLOSE": "hub1_CLOSE"})
    hub2 = hub2.rename(columns={"CLOSE": "hub2_CLOSE"})

    hub1_hub2_diff = pd.DataFrame(hub1["hub1_CLOSE"] - hub2["hub2_CLOSE"], columns=["hub1_hub2_diff"], index=hub1.index)

    for i in range(window_size, window_size + 7):
        hub1[f"hub1_CLOSE-{i-window_size}"] = hub1["hub1_CLOSE"].shift(i)
        hub2[f"hub2_CLOSE-{i - window_size}"] = hub2["hub2_CLOSE"].shift(i)
        hub1_hub2_diff[f"hub1_hub2_diff-{i - window_size}"] = hub1_hub2_diff["hub1_hub2_diff"].shift(i)

    # Concatenate and drop NaN rows in one step
    data = pd.concat([hub1, hub2, hub1_hub2_diff], axis=1).dropna()


    features = [
        'hub1_CLOSE-1', #'hub1_CLOSE-2', 'hub1_CLOSE-3', 'hub1_CLOSE-4', 'hub1_CLOSE-5', 'hub1_CLOSE-6',
        'hub2_CLOSE-1', #'hub2_CLOSE-2', 'hub2_CLOSE-3', 'hub2_CLOSE-4', 'hub2_CLOSE-5', 'hub2_CLOSE-6',
        'hub1_hub2_diff-0', 'hub1_hub2_diff-1', 'hub1_hub2_diff-2', 'hub1_hub2_diff-3', 'hub1_hub2_diff-4', 'hub1_hub2_diff-5', 'hub1_hub2_diff-6'
    ]

    X = data[features].values
    y = data[['hub1_CLOSE', 'hub2_CLOSE']].values

    X_train, X_test = X[:-test_size], X[-test_size:]
    y_train, y_test = y[:-test_size], y[-test_size:]
    X_train, X_val = X_train[:-validation_size], X_train[-validation_size:]
    y_train, y_val = y_train[:-validation_size], y_train[-validation_size:]

    return X_train, y_train, X_val, y_val, X_test, y_test

def objective(trial):
    # Load data
    X_train, y_train, X_val, y_val, X_test, y_test = load_and_prepare_data(
        hub1_name=hub1_name, hub2_name=hub2_name, 
        window_size=5, test_size=250, validation_size=250
    )

    # Build model
    model = build_model(trial)

    batch_size = trial.suggest_int("batch_size", 2,4, step=2)

    # Train model
    model.fit(X_train, y_train, epochs=25, batch_size=batch_size, 
              validation_data=(X_val, y_val), verbose=0, shuffle=False)

    # Evaluate on validation set
    val_loss = model.evaluate(X_val, y_val, verbose=0)
    return val_loss

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# Display best trial
print("Best trial:")
trial = study.best_trial
print(f"Value (MAPE): {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Evaluate best model on test set
best_model = build_model(study.best_trial)
X_train, y_train, X_val, y_val, X_test, y_test = load_and_prepare_data(
    hub1_name=hub1_name, hub2_name=hub2_name, window_size=5, test_size=250, validation_size=250
)
keras.utils.set_random_seed(42)
best_batch_size = study.best_trial.params["batch_size"]
best_model.fit(np.vstack([X_train, X_val]), np.vstack([y_train, y_val]), shuffle = False, epochs=25, batch_size=best_batch_size, verbose=0)
predictions = best_model.predict(X_test)

# Calculate and print MAPE for each hub
mape_hub1 = mean_absolute_percentage_error(y_test[:, 0], predictions[:, 0]) * 100
mape_hub2 = mean_absolute_percentage_error(y_test[:, 1], predictions[:, 1]) * 100
print(f"MAPE for {hub1_name}: {mape_hub1:.2f}%")
print(f"MAPE for {hub2_name}: {mape_hub2:.2f}%")


[I 2024-10-30 16:59:24,163] A new study created in memory with name: no-name-694e848d-adde-4223-8158-e184097c3642
[I 2024-10-30 16:59:35,434] Trial 0 finished with value: 11.255555152893066 and parameters: {'units_0': 2, 'learning_rate': 0.0016299789780296827, 'batch_size': 2}. Best is trial 0 with value: 11.255555152893066.
[I 2024-10-30 16:59:46,732] Trial 1 finished with value: 14.031111717224121 and parameters: {'units_0': 2, 'learning_rate': 0.0030795676314273866, 'batch_size': 2}. Best is trial 0 with value: 11.255555152893066.
[I 2024-10-30 16:59:53,570] Trial 2 finished with value: 14.627769470214844 and parameters: {'units_0': 2, 'learning_rate': 0.00563858375827385, 'batch_size': 4}. Best is trial 0 with value: 11.255555152893066.
[I 2024-10-30 17:00:00,687] Trial 3 finished with value: 16.94601821899414 and parameters: {'units_0': 2, 'learning_rate': 0.0067717188549123826, 'batch_size': 4}. Best is trial 0 with value: 11.255555152893066.
[I 2024-10-30 17:00:12,314] Trial 4 f

Best trial:
Value (MAPE): 10.934887886047363
Params:
    units_0: 2
    learning_rate: 0.0021246305313405733
    batch_size: 2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
MAPE for ttf: 6.11%
MAPE for nbp: 6.86%


In [221]:
X_train[0:10]

array([[27.218     , 28.39325622, -1.17525622, -1.36686785, -1.25789822,
        -1.31696325, -1.18920841, -1.4341256 ],
       [26.835     , 28.04845395, -1.21345395, -1.17525622, -1.36686785,
        -1.25789822, -1.31696325, -1.18920841],
       [26.22      , 27.39920497, -1.17920497, -1.21345395, -1.17525622,
        -1.36686785, -1.25789822, -1.31696325],
       [25.759     , 26.9094428 , -1.1504428 , -1.17920497, -1.21345395,
        -1.17525622, -1.36686785, -1.25789822],
       [26.628     , 28.02107415, -1.39307415, -1.1504428 , -1.17920497,
        -1.21345395, -1.17525622, -1.36686785],
       [26.625     , 27.99749411, -1.37249411, -1.39307415, -1.1504428 ,
        -1.17920497, -1.21345395, -1.17525622],
       [26.328     , 27.43588221, -1.10788221, -1.37249411, -1.39307415,
        -1.1504428 , -1.17920497, -1.21345395],
       [25.942     , 27.11229399, -1.17029399, -1.10788221, -1.37249411,
        -1.39307415, -1.1504428 , -1.17920497],
       [26.025     , 27.0848398 